# COMP/LING 445/645

# 5, 6 - Formal Languages:

We want to built a computational model for language with some sort of system of grammaticality. 

We wish to denote what is in or out of our language!

Remember Basic Set Theory. A **formal language** is described by an Alphabet, or in this class, denoted as a **Vocabulary**. Formal Languages are closed under **Union**, and **Power Set**. The prof introduces Intersection and Difference but that doesn't necessarily imply clojure (HAH).

In [56]:
(defn prefix? [pr str]
  (if (> (count pr) (count str))
    false
    (if (empty? pr)
      true
      (if (= (first pr) (first str))
        (prefix? (rest pr) (rest str))
        false))))

#'user/prefix?

In [57]:
(prefix? '(a b c) '(a b))

false

In out observations, natural languages are not necesarily finite. In class we proved that the finite class _FIN_ was not able to model the recursive nature of some languages, due to it being closed _under homomorphism_. We need to be able to build a model which can accomadate the unbounded recursive quality natural language may have.

# 7 - Infinite Languages

Generators and recognisers are ways to model languges sans probability. These are good for yes/no inclusions of examples of our languge. While samplers and scorers allow for probabilistic based modelling. 


## Language Recognisers


In [58]:
(defn lang-ab*? [str]
  (if (empty? str)
    true
    (if (prefix? '(a b) str)
      (lang-ab*? (rest (rest str)))
      false)))


#'user/lang-ab*?

In [59]:
(lang-ab*? '(ab))   

false

In [60]:
(lang-ab*? '(a b))

true

In [61]:
(lang-ab*? '())

true

## Language Generators

Note that this cannot generate the **whole** set for the (ab)* star language, but provides a bijective mapping from all natural number to our language. 

In [62]:
(defn generate-abn [n]
    (if (= n 0)
        '()
        (concat '(a b) (generate-abn (- n 1)))))

#'user/generate-abn

In [63]:
(generate-abn 13)

(a b a b a b a b a b a b a b a b a b a b a b a b a b)

## Generative models of language

Uses generators as models of linguistic structure. The important thing is that we will try to define finite programs that characterize the set of possible English sentences

# 8 - Distributions over languages

We wish to implement more than just a yes no denomination for string being in our language. We would also like to model how common a string is in our language, or how likely it is to occur in some corpus. 

## Computational models of formal languages

When a language is finite we can specify its probability distribution via a single probability vector $\theta$

However when a langugage is infinite we must define its probability through some corpus. We will be doing this through some _probabilistic generative model_. this is just a generator which samples based on some probability distribution

In [64]:
(defn generate-ab* [n]
  (if (= n 0)
    '()
    (concat '(a b) (generate-ab*  (- n 1)))))

(generate-ab*  26)

(a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b)

Note how this determines _(ab)*_ based on some non-deterministic value we specify. Lets do this now with a non-deterministic random choice: Let's say a coin flip

In [65]:
(defn coinflip []
    (if (> (rand 1) 0.5)
        true
        false))

(coinflip)

true

In [66]:
(frequencies (repeatedly 1000000 coinflip))

{true 499841, false 500159}

Now lets use coinflip to sample non-negative numbers

In [67]:
(defn sample-flip []
    (if (coinflip)
        0
        (+ 1 (sample-flip))))

(sample-flip)

2

In [68]:
(frequencies (repeatedly 1000000 sample-flip))

{0 499464, 7 3913, 1 250176, 4 31529, 15 12, 13 65, 6 7917, 17 3, 3 62463, 12 96, 2 124904, 19 1, 11 238, 9 969, 5 15786, 14 38, 16 6, 10 507, 8 1913}

Lets use this theta distribution to generate random samples for our language _(ab)*_

In [69]:
(defn sample-ab* [] (generate-ab* (sample-flip)))
(sample-ab*)

(a b a b)

In [70]:
(frequencies (repeatedly 1000000 sample-ab*))

{(a b a b a b a b) 31194, (a b a b a b a b a b a b a b a b) 1916, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 4, (a b a b a b a b a b a b a b a b a b a b a b a b) 133, (a b a b a b a b a b a b a b) 3981, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 4, () 500166, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 32, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 1, (a b a b) 125076, (a b a b a b a b a b a b a b a b a b a b) 457, (a b a b a b a b a b a b a b a b a b a b a b a b a b) 63, (a b a b a b a b a b a b) 7852, (a b a b a b) 62464, (a b) 249796, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 1, (a b a b a b a b a b a b a b a b a b) 939, (a b a b a b a b a b) 15669, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 1, (a b a b a b a b a b a b a b a b a b a b a b a b a b a b a b) 9, (a b a b a b a b a b a b a b a b a b a b a b) 242}

Note that this simple distribution is actually the geometric series! 
In this case q = 0.5
Now let us introduce scorers, which will take in a string and output its probability based on the theta we defined above

In [71]:
(defn score-ab* [str]
  (if (empty? str)
    0.5
    (if (prefix? '(a b) str)
      (* 0.5 (score-ab* (rest (rest str))))
      0)))

(score-ab* '(a b a b))

0.125

Note how this relation is just the inverse of sampling

# 9 - Models and data

With infinite languages it is impossible to generate a set of all posible sentences. so let us build a subset of our language (a corpus) based on frequencies of occurance. With these we can build models based on language, that with the probabilities, we can measure some metric to evaluate it based on _goodness of fit_ in relation to real life use of our language. 

Let us utilize our previously described sampler to generate a nice corpus

In [72]:
(defn sample-corpus [generator size]
  (if (= size 0)
    '()
    (cons (generator)
          (sample-corpus generator (- size 1)))))

(sample-corpus sample-ab* 4)

((a b a b) () () ())

Now let us also use our scorer to score our corpus. Note that this value is the just probability of our sampler generating this corpus. 

In [78]:
(def ex-corpus (sample-corpus sample-ab* 10))
ex-corpus

(() (a b) (a b) (a b) (a b) (a b a b) (a b a b) () (a b a b a b a b a b a b) ())

In [ ]:
(defn score-corpus [scorer corpus]
  (if (empty? corpus)
    1.0
    (* (scorer (first corpus))
       (score-corpus scorer (rest corpus)))))
(score-corpus score-ab* e)

## Fit of a Language to a Corpus 

How can we tell out of two probabilistic models which is a better representation to our corpus?

For our corpus _ex-corpus_ lets us define a new model based on the language 
_{a, b}*_ to describe it

In [85]:
(defn sample-a*b* []
    (if (coinflip)
        '()
        (cons (if (coinflip) 'a 'b)
              (sample-a*b*))))
(sample-a*b*)

(b)

In [87]:
(frequencies (repeatedly 1000000 sample-a*b*))

{(a b b b b a) 141, (a a b a a a b b b) 2, (a b b a a b a a b b) 2, (b b a b a b a a b a a) 1, (a a b a a a b a b a a a b b) 1, (b b a b a b a b b) 5, (a b a a b a b b a a b) 1, (b a a b b a a a b a) 3, (b b b a a a a b b) 2, (a a b b b a b b a) 2, (a a a b b a b a b b a b b b) 1, (b b b a a a a) 35, (b b a b a a a a b a) 1, (a b a b a b a b) 7, (b a a a b b a b a b) 2, (b a a a a a a a b b) 1, (b b b a b a b a b) 1, (a b a a b b b b b b) 1, (b b a b a a a a a) 1, (a a b b a b b b a) 3, (b b a b b b a a b) 3, (b b a b b b b a b a a) 1, (a a a b a a a a b) 1, (a b b a b b b a a b) 2, (a a a a b a b a b) 4, (b a a a b b a) 33, (b a b b a a a b b) 2, (b b a b b a b b a a) 1, (a b b a b a a a a) 5, (a a a a a b a b b a) 1, (b b b b a a a b a b b) 1, (b a a a b a a a) 6, (b a a b a b a b b) 2, (b b a b a a a a a b) 1, (b b a a b a a b b a) 2, (a b b a a b b b) 14, (b a a a b b a a b a) 1, (a b b b a b a a b a b a b a) 1, (b a a) 7863, (a a a b b) 453, (a a b a a a b a b b) 3, (b a a a b b b

Wow that looks awful! Now lets do the inverse! Implement a scorer. 

In [90]:
(defn score-a*b* [str]
  (if (empty? str)
    0.5
    (* (if (= (first str) 'a)
         0.5
         0.5)
       (score-a*b* (rest str)))))
(list
(score-a*b* '(a b a b))
(score-ab* '(a b a b)))

(0.03125 0.125)

In [91]:
(list
(score-a*b* ex-corpus)
(score-ab* ex-corpus))

(4.8828125E-4 0)

Note how much smaller (In some cases zero) {a, b}* models our ex-corpus. This proves that the (ab) model fits our corpus better 